1. Suppose a student has taken an education load of size Rs 0.8 million. The
interest rate is 12%. Write a python program that generates the schedule of
repayment of the loan in 5 years (or ‘n’ number of years). Assume the first
payment date is 01 January 2026. Also, show the breakup of each payment is
principal and interest.

Step 1: Importing Neccesey libraries

In [1]:
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

The EMI is calculated using the annuity formula:

$$
EMI = P \times r \times \frac{(1 + r)^n}{(1 + r)^n - 1}
$$

Where:  
- \( P \) = Principal loan amount  
- \( r \) = Monthly interest rate (annual interest rate divided by 12)  
- \( n \) = Total number of monthly payments


In [2]:
# Function to generate loan repayment schedule
def generate_loan_schedule(principal, annual_rate, years, start_date_str):
    monthly_rate = (annual_rate / 12) / 100  # Monthly interest rate
    n_months = years * 12  # Total number of payments
    
    # Calculate EMI using the annuity formula
    emi = principal * monthly_rate * (1 + monthly_rate)**n_months / ((1 + monthly_rate)**n_months - 1)
    
    schedule = []
    balance = principal
    start_date = datetime.strptime(start_date_str, "%d-%m-%Y")
    
    for i in range(1, n_months + 1):
        interest = balance * monthly_rate
        principal_payment = emi - interest
        balance -= principal_payment
        payment_date = start_date + pd.DateOffset(months=i-1)
        
        schedule.append({
            'Payment No': i,
            'Payment Date': payment_date.strftime("%d-%b-%Y"),
            'EMI': round(emi, 2),
            'Principal Paid': round(principal_payment, 2),
            'Interest Paid': round(interest, 2),
            'Outstanding Balance': round(balance if balance > 0 else 0, 2)
        })
    
    return pd.DataFrame(schedule)

In [3]:
df_schedule = generate_loan_schedule(
    principal=800000, 
    annual_rate=12, 
    years=5, 
    start_date_str="01-01-2026"
)

In [15]:
print(df_schedule.to_string(index=False))

 Payment No Payment Date      EMI  Principal Paid  Interest Paid  Outstanding Balance
          1  01-Jan-2026 17795.56         9795.56        8000.00            790204.44
          2  01-Feb-2026 17795.56         9893.51        7902.04            780310.93
          3  01-Mar-2026 17795.56         9992.45        7803.11            770318.48
          4  01-Apr-2026 17795.56        10092.37        7703.18            760226.11
          5  01-May-2026 17795.56        10193.30        7602.26            750032.81
          6  01-Jun-2026 17795.56        10295.23        7500.33            739737.58
          7  01-Jul-2026 17795.56        10398.18        7397.38            729339.40
          8  01-Aug-2026 17795.56        10502.16        7293.39            718837.23
          9  01-Sep-2026 17795.56        10607.19        7188.37            708230.05
         10  01-Oct-2026 17795.56        10713.26        7082.30            697516.79
         11  01-Nov-2026 17795.56        10820.39     

2. Consider a 20-years 8% bond with the coupon paid semi-annually. What will be
the present value of the bond? Use the following spot rates tables to compute
the present value. Also, compute duration and convexity.


### **Given Data**
| Period | Spot Rate (Annual) |
|--------|--------------------|
| 6M     | 2.90%              |
| 1 Yr   | 4.40%              |
| 2 Yr   | 4.80%              |
| 3 Yr   | 5.00%              |
| 5 Yr   | 5.30%              |
| 10 Yr  | 5.40%              |
| 20 Yr  | 5.50%              |

### Bond Valuation Formulas

#### 1. Semi-Annual Coupon Payment
$$
C = \frac{\text{Coupon Rate} \times \text{Face Value}}{2}
$$

---

#### 2. Linear Interpolation of Spot Rate
If the spot rate for the desired period \( t \) is not directly given:
$$
r_t = r_0 + \left( \frac{r_1 - r_0}{t_1 - t_0} \right)(t - t_0)
$$
Where:  
- \( r_0 \) and \( r_1 \) are the spot rates at the nearest lower and higher time points \( t_0 \) and \( t_1 \), respectively.

---

#### 3. Discount Factor (for semi-annual compounding)
$$
DF_t = \frac{1}{\left(1 + \frac{r_t}{2} \right)^{2t}}
$$

---

#### 4. Present Value of the Bond
$$
PV = \sum_{t=1}^{T} \left( CF_t \times DF_t \right)
$$
Where:  
- \( CF_t \) is the cash flow at time \( t \)  
- \( DF_t \) is the corresponding discount factor

---

#### 5. Macaulay Duration
$$
D = \frac{ \sum_{t=1}^{T} \left( t \times CF_t \times DF_t \right)}{PV}
$$

---

#### 6. Convexity
$$
\text{Convexity} = \frac{ \sum_{t=1}^{T} \left( CF_t \times DF_t \times t \times \left( t + \frac{1}{2} \right) \right)}{PV \times \left(1 + \frac{r_T}{2} \right)^2}
$$
Where \( r_T \) is the spot rate at maturity.


In [37]:
# Bond parameters
face_value = 100
coupon_rate = 0.08  # 8% annual
semi_annual_coupon = coupon_rate * face_value / 2
years_to_maturity = 20
periods = years_to_maturity * 2

# Given spot rates (period in years, rate in %)
spot_rates = {
    0.5: 0.029,   # 6M
    1: 0.044,     # 1Y
    2: 0.048,     # 2Y
    3: 0.05,      # 3Y
    5: 0.053,     # 5Y
    10: 0.054,    # 10Y
    20: 0.055     # 20Y
}

# Function to interpolate spot rates for any period
def get_spot_rate(period):
    if period <= 0.5:
        return spot_rates[0.5]
    elif period >= 20:
        return spot_rates[20]
    
    # Find the nearest lower and higher periods
    lower_periods = [p for p in spot_rates.keys() if p <= period]
    higher_periods = [p for p in spot_rates.keys() if p > period]

    if not lower_periods or not higher_periods:
        # Extrapolate if necessary
        if not lower_periods:
            return spot_rates[min(spot_rates.keys())]
        else:
            return spot_rates[max(spot_rates.keys())]
    
    lower_period = max(lower_periods)
    higher_period = min(higher_periods)
    
    # Linear interpolation
    x = period
    x0, y0 = lower_period, spot_rates[lower_period]
    x1, y1 = higher_period, spot_rates[higher_period]
    
    return y0 + (x - x0) * (y1 - y0) / (x1 - x0)

# Calculate cash flows and discount factors
cash_flows = [semi_annual_coupon] * periods
cash_flows[-1] += face_value  # Add principal at maturity

period_times = [(i+1)/2 for i in range(periods)]  # in years
spot_rates_list = [get_spot_rate(t) for t in period_times]
print(f"Spot Rates: ")
print([round(rate, 4) for rate in spot_rates_list],"\n") 

discount_factors = [1 / (1 + r/2)**(2*t) for r, t in zip(spot_rates_list, period_times)]


# Present Value calculation
present_value = sum(cf * df for cf, df in zip(cash_flows, discount_factors))

# Duration calculation
weighted_pv = [cf * df * t for cf, df, t in zip(cash_flows, discount_factors, period_times)]
macaulay_duration = sum(weighted_pv) / present_value

# Convexity calculation
convexity = sum([cf * df * t * (t + 1) for cf, df, t in zip(cash_flows, discount_factors, period_times)]) / (present_value * (1 + spot_rates_list[-1]/2)**2)

print(f"For the bond with face value {face_value} and coupon rate {coupon_rate*100}%:")
print(f"Present Value of the Bond: {present_value:.4f}")
print(f"Macaulay Duration: {macaulay_duration:.4f} years")
print(f"Convexity: {convexity:.4f}")

Spot Rates: 
[0.029, 0.044, 0.046, 0.048, 0.049, 0.05, 0.0508, 0.0515, 0.0522, 0.053, 0.0531, 0.0532, 0.0533, 0.0534, 0.0535, 0.0536, 0.0537, 0.0538, 0.0539, 0.054, 0.0541, 0.0541, 0.0541, 0.0542, 0.0542, 0.0543, 0.0544, 0.0544, 0.0544, 0.0545, 0.0546, 0.0546, 0.0546, 0.0547, 0.0548, 0.0548, 0.0549, 0.0549, 0.0549, 0.055] 

For the bond with face value 100 and coupon rate 8.0%:
Present Value of the Bond: 131.0657
Macaulay Duration: 11.4347 years
Convexity: 180.8706


### 3. Complete the following tasks under this assignment.
   
a. Download one-year INFY, and RIL stock prices from National Stock
Exchange

b. Compute the daily returns for both.

c. Fit the return series separately to Generalized Gaussian distribution if you
can or fit the normal distribution.

d. Fit the T-Copula with the return series. You may use the `copulae`
package for the same or your own.

e. Construct a portfolio with equal units from both.

f. Simulate 10,000 or more random returns of your portfolio using copula

g. And calculate the maximum loss you may have in a day that you can say
with 95% confidence

a. Download 1-year INFY & RIL stock prices from NSE


In [21]:
# pip install yfinance


In [3]:
def get_data(my_symbol,start_date = '2000-01-01',end_date=None):
    import yfinance as yahooFinance
    from datetime import date
    my_finance_object = yahooFinance.Ticker(my_symbol)
    if end_date==None:
        today = date.today()
    else:
        today = end_date
    my_df = my_finance_object.history(start=start_date, end=today)
    return my_df

In [30]:
start = "2024-04-01"
end = "2025-03-31"
infy = get_data("INFY", start_date=start, end_date=end)
ril = get_data("RELIANCE.NS", start_date=start, end_date=end)
ril.index = ril.index.date
infy.index = infy.index.date

In [39]:
infy.info(),ril.info()

<class 'pandas.core.frame.DataFrame'>
Index: 250 entries, 2024-04-01 to 2025-03-28
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          250 non-null    float64
 1   High          250 non-null    float64
 2   Low           250 non-null    float64
 3   Close         250 non-null    float64
 4   Volume        250 non-null    int64  
 5   Dividends     250 non-null    float64
 6   Stock Splits  250 non-null    float64
dtypes: float64(6), int64(1)
memory usage: 15.6+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 248 entries, 2024-04-01 to 2025-03-28
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          248 non-null    float64
 1   High          248 non-null    float64
 2   Low           248 non-null    float64
 3   Close         248 non-null    float64
 4   Volume        248 non-null    int64  
 5   Dividends     248 non-nul

(None, None)

In [40]:
ril.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
2024-04-01,1492.474976,1493.974976,1482.500000,1484.775024,5013880,0.0,0.0
2024-04-02,1484.000000,1494.000000,1475.000000,1486.949951,8910166,0.0,0.0
2024-04-03,1482.074951,1484.449951,1468.900024,1471.599976,7008292,0.0,0.0
2024-04-04,1479.750000,1479.750000,1450.000000,1462.925049,14490272,0.0,0.0
2024-04-05,1460.875000,1470.800049,1456.000000,1460.099976,7442298,0.0,0.0
